In [1]:
%matplotlib inline
from galaxy_analysis.plot.plot_styles import *
####
import numpy as np
import matplotlib.pyplot as plt
from galaxy_analysis.analysis import Galaxy
from sklearn.decomposition import PCA
import yt

/home/aemerick/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/aemerick/code/galaxy_analysis/particle_analysis/IMF.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aemerick/anaconda2/lib/python2.7/sit

/home/aemerick/code/galaxy_analysis/particle_analysis/sn_rate.py:4: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", li

/home/aemerick/code/galaxy_analysis/plot/plot_styles.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/aemerick/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aemerick/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, i

In [6]:
#
# Load the data set and get all defined abundance ratios
#
gal = Galaxy('DD0619', wdir="/home/aemerick/work/enzo_runs/sn_H2atten_H2sh/")
derived_fields = gal.ds.derived_field_list
all_abundance_fields = [x[1] for x in derived_fields if (('particle_' in x[1]) and ('_over_' in x[1]))]

Parsing Hierarchy : 100%|██████████| 422/422 [00:00<00:00, 46235.73it/s]


In [6]:
gal.df['particle_O_fraction']

YTArray([3.98402921e-28, 3.98402921e-28, 1.73634694e-11, ...,
         2.67069546e-09, 2.67069546e-09, 9.50525643e-06]) (dimensionless)

In [7]:
#
# Now gather all of the particle data for all main sequence stars:
#
select = (gal.df['creation_time'] > 100.0*yt.units.Myr) * (gal.df['creation_time'] < 1000.0*yt.units.Myr)

# for testing
#abundance_fields = ['particle_O_over_Fe','particle_Mg_over_Fe','particle_N_over_Fe','particle_Ba_over_Fe']

abundance_fields = [x for x in all_abundance_fields if '_over_Fe' in x]

dtype = [ (x,'f8') for x in abundance_fields ]

get_field = lambda name : gal.df[('io',name)][select].value
all_data  = np.array( map( get_field, abundance_fields )) # , dtype = dtype)
#all_data = np.zeros( len(gal.df['particle_type'][select]) , dtype = {'names':abundance_fields, 'formats':['f8']*len(abundance_fields)})
#
#for field in abundance_fields:
#    all_data[field] = get_field(field)
print np.shape(all_data)

(16, 19259)


In [8]:
from sklearn.preprocessing import StandardScaler
scaled_data = StandardScaler().fit_transform(all_data)

In [9]:
pca = PCA(n_components=len(abundance_fields))
pca.fit(scaled_data)
print pca.explained_variance_ratio_

[7.81560980e-01 1.62047757e-01 3.18037391e-02 1.55734024e-02
 3.89135457e-03 1.97210217e-03 1.10958169e-03 7.87511807e-04
 6.70461618e-04 3.00246758e-04 2.36360929e-04 2.57211760e-05
 1.75243110e-05 3.17111413e-06 8.55731294e-08 5.37435001e-32]


In [104]:
pca.score(

/home/aemerick/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/base.py:73: RuntimeWarning: divide by zero encountered in divide
  precision = np.dot(components_, components_.T) / self.noise_variance_
/home/aemerick/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/base.py:73: RuntimeWarning: invalid value encountered in divide
  precision = np.dot(components_, components_.T) / self.noise_variance_


ValueError: array must not contain infs or NaNs

In [51]:
order = []
for i, x in enumerate(all_data):
    for y in top_five:
        if np.allclose(x,y):
            order += [i]

In [52]:
top_five

array([[ 1.86140727e-02,  1.86140727e-02, -7.67567152e-03, ...,
        -5.82941054e-03, -5.82941054e-03, -8.44704018e-03],
       [ 5.57655910e-03,  5.57655910e-03,  1.00452428e-02, ...,
         1.38801371e-02,  1.38801371e-02, -2.77208113e-03],
       [-1.06244542e-02, -1.06244542e-02, -4.58814744e-02, ...,
         1.80957185e-04,  1.80957185e-04, -4.82401029e-04],
       [-5.31198295e-01,  1.74699972e-02,  8.44005456e-01, ...,
        -5.93670436e-04,  1.03935126e-04, -1.38739042e-04]])

In [54]:
PCA.transform?